In [1]:
import numpy as np
import math
from scipy import integrate
import matplotlib.pyplot as plt
import random
import time
from scipy.stats import rankdata

In [2]:
def lap_noise(data, epsilon, K, N, M):
    pri_data = data + np.random.laplace(loc = 0.0, scale = (K*8*(N-1)/N)/epsilon, size=data.shape)
    
    min = 10000
    
    for i in range(M):
        if pri_data[i] > 0 and pri_data[i] < min:
            min = pri_data[i]
    
    for i in range(M):
        if pri_data[i] < 0:
            pri_data[i] = min
    
    return pri_data

In [3]:
def hs(n):
    h = n[1]+n[2]+n[3] + 2*(n[4]+n[5]+n[6]+n[7]+n[8]+n[9])
    i = n[3]+n[7]+n[8] + 2*n[9]
    j = n[2]+n[5] + 2*n[6] + n[8]
    
    if h == 0:
        return 0
    else:
        return ((2*i+2*j-h)**2)/h

def SHD_hs(cc,n):
    h = n[1]+n[2]+n[3] + 2*(n[4]+n[5]+n[6]+n[7]+n[8]+n[9])
    i = n[3]+n[7]+n[8] + 2*n[9]
    j = n[2]+n[5] + 2*n[6] + n[8]
    T = hs(n)
    
    d = np.zeros(2)
    N = np.zeros(10)
    
    for k in range(10):
        N[k] = n[k]
    
    if T < cc:
        while T < cc:
            if N[4] > 0: N[4] -= 1
            elif N[1] > 0: N[1] -= 1
            elif N[5] > 0: N[5] -= 1
            elif N[7] > 0: N[7] -= 1
            elif N[0] > 0: N[0] -= 1
            elif N[2] > 0: N[2] -= 1
            else: N[3] -= 1
            N[9] += 1
            
            T = hs(N)
            d[0] -= 1
        
        T = hs(n)
        for k in range(10):
            N[k] = n[k]
        
        while T < cc:
            if N[6] > 0: N[6] -= 1
            elif N[8] > 0: N[8] -= 1
            elif N[9] > 0: N[9] -= 1
            elif N[2] > 0: N[2] -= 1
            elif N[3] > 0: N[3] -= 1
            elif N[5] > 0: N[5] -= 1
            elif N[7] > 0: N[7] -= 1
            elif N[0] > 0: N[0] -= 1
            else: N[1] -= 1
            N[4] += 1
            
            T = hs(N)
            d[1] -= 1
        
        if d[0] > d[1]:
            return d[0]
        else:
            return d[1]
        
    else:
        step = 0
        
        for k in range(10):
            N[k] = n[k]
            
        if i+j > h/2:
            while T >= cc:
                if N[6] > 0: N[6] -= 1
                elif N[8] > 0: N[8] -= 1
                elif N[9] > 0: N[9] -= 1
                elif N[2] > 0: N[2] -= 1
                elif N[3] > 0: N[3] -= 1
                elif N[0] > 0: N[0] -= 1
                elif N[5] > 0: N[5] -= 1
                elif N[7] > 0: N[7] -= 1
                else: N[1] -= 1
                N[4] += 1
                
                T = hs(N)
                step += 1
        else:
            while T >= cc:
                if N[4] > 0: N[4] -= 1
                elif N[1] > 0: N[1] -= 1
                elif N[0] > 0: N[0] -= 1
                elif N[5] > 0: N[5] -= 1
                elif N[7] > 0: N[7] -= 1
                elif N[2] > 0: N[2] -= 1
                else: N[3] -= 1
                N[9] += 1
                
                T = hs(N)
                step += 1
                
        return step-1

In [4]:
def appx_SHD_hs(cc,n):
    h = n[1]+n[2]+n[3] + 2*(n[4]+n[5]+n[6]+n[7]+n[8]+n[9])
    i = n[3]+n[7]+n[8] + 2*n[9]
    j = n[2]+n[5] + 2*n[6] + n[8]
    T = hs(n)
    
    if T < cc:
        if i+j >= h/2:
            if h <= cc:
                return -math.ceil((cc-(i+j))/2)
            else:
                return -math.ceil(((h+math.sqrt(h*cc))/2 - (i+j))/2)
        else:
            if h <= cc:
                return -math.ceil((cc-h+i+j)/2)
            else:
                return -math.ceil((i+j - (h-math.sqrt(h*cc))/2)/2)
    else:
        if i+j >= h/2:
            return math.ceil((i+j - (h+math.sqrt(h*cc))/2)/2) - 1
        else:
            return math.ceil(((h-math.sqrt(h*cc))/2 - (i+j))/2)-1

In [5]:
def random_num(pd):
    dist = np.cumsum(pd).tolist()
    dist[-1] = 1.0
    num = np.random.rand()
    dist.append(num)
    return sorted(dist).index(num)

def exp_mec(shd, m, K, epsilon):
    S = np.zeros(K)
    k = 0
    
    w = np.zeros(m)
    p = np.zeros(m)
    
    while k < K:
        sumw = 0
        for i in range(m):
            w[i] = math.exp(epsilon * shd[i] / (2*K))
            sumw += w[i]
        for i in range(m):
            p[i] = w[i]/sumw
        
        x = random_num(p)
        shd[x] = -1000000.0
        S[k] = x
        k += 1
    
    return S

In [6]:
for j in range(5):
    N = 150
    M = 5000
    H = np.zeros(M)
    I = np.zeros(M)
    J = np.zeros(M)
    
    n = np.zeros((M,10))
    
    for i in range(M-10):
        n[i][4] = np.random.binomial(2*N, 1/5)
        S = np.random.binomial(2*N-n[i][4], 1/4) #n7+n9+n10
        n[i][6] = np.random.binomial(S, 1/3)
        n[i][8] = np.random.binomial(S-n[i][6], 1/3)
        n[i][9] = S-n[i][6]-n[i][8]
        n[i][1] = np.random.binomial(2*N-n[i][4]-S, 1/3)
        T = np.random.binomial(2*N-n[i][4]-n[i][1]-S, 1/2) #n3+n4
        n[i][2] = np.random.binomial(T, 1/2)
        n[i][3] = T - n[i][2]
        R = 2*N-n[i][4]-n[i][1]-S-T
        n[i][0] = np.random.binomial(R, 1/3)
        n[i][5] = np.random.binomial(R-n[i][0], 1/2)
        n[i][7] = R - n[i][0] - n[i][5]
        
        H[i] = n[i][1]+n[i][2]+n[i][3] + 2*(n[i][4]+n[i][5]+n[i][6]+n[i][7]+n[i][8]+n[i][9])
        I[i] = n[i][3]+n[i][7]+n[i][8] + 2*n[i][9]
        J[i] = n[i][2]+n[i][5] + 2*n[i][6] + n[i][8]
    
    for i in range(M-10,M):
        n[i][4] = np.random.binomial(2*N, 1.5/5)
        S = np.random.binomial(2*N-n[i][4], 1/4) #n7+n9+n10
        n[i][6] = np.random.binomial(S, 1/3)
        n[i][8] = np.random.binomial(S-n[i][6], 1/3)
        n[i][9] = S-n[i][6]-n[i][8]
        n[i][1] = np.random.binomial(2*N-n[i][4]-S, 1.5/3)
        T = np.random.binomial(2*N-n[i][4]-n[i][1]-S, 1/2) #n3+n4
        n[i][2] = np.random.binomial(T, 1/2)
        n[i][3] = T - n[i][2]
        R = 2*N-n[i][4]-n[i][1]-S-T
        n[i][0] = np.random.binomial(R, 1/3)
        n[i][5] = np.random.binomial(R-n[i][0], 1/2)
        n[i][7] = R - n[i][0] - n[i][5]
        
        H[i] = n[i][1]+n[i][2]+n[i][3] + 2*(n[i][4]+n[i][5]+n[i][6]+n[i][7]+n[i][8]+n[i][9])
        I[i] = n[i][3]+n[i][7]+n[i][8] + 2*n[i][9]
        J[i] = n[i][2]+n[i][5] + 2*n[i][6] + n[i][8]
        
    stats = np.zeros(M)
    
    for i in range(M):
        if H[i] == 0:
            stats[i] = 0
        else:
            stats[i] = ((2*I[i] + 2*J[i] - H[i])**2)/H[i]
    
    shd = np.zeros(M)
    appx_shd = np.zeros(M)
    cc = 19.5
    epsilon = 3
    K = 3
    
    lap_start = time.time()
    lap_noise(stats, epsilon, K, N, M)
    lap_end = time.time()
    
    print("lap_time =",lap_end-lap_start,"[sec]")
    
    appx_start = time.time()
    for i in range(M):
        k = [int(n[i][0]), int(n[i][1]), int(n[i][2]), int(n[i][3]), int(n[i][4]), int(n[i][5]), int(n[i][6]), int(n[i][7]), int(n[i][8]), int(n[i][9])]
        appx_shd[i] = appx_SHD_hs(cc,k)
    exp_mec(appx_shd, M, K, epsilon)
    appx_end = time.time()
    
    print("appx_time =",appx_end-appx_start,"[sec]")
    
    exact_start = time.time()
    for i in range(M):
        k = [int(n[i][0]), int(n[i][1]), int(n[i][2]), int(n[i][3]), int(n[i][4]), int(n[i][5]), int(n[i][6]), int(n[i][7]), int(n[i][8]), int(n[i][9])]
        shd[i] = SHD_hs(cc,k)
    exp_mec(shd, M, K, epsilon)
    exact_end = time.time()
    
    print("exact_time =",exact_end-exact_start,"[sec]")
    print("----------")

lap_time = 0.003751039505004883 [sec]
appx_time = 0.05032706260681152 [sec]
exact_time = 1.6552221775054932 [sec]
----------
lap_time = 0.003674030303955078 [sec]
appx_time = 0.04819989204406738 [sec]
exact_time = 1.6475780010223389 [sec]
----------
lap_time = 0.0036323070526123047 [sec]
appx_time = 0.04860496520996094 [sec]
exact_time = 1.6541321277618408 [sec]
----------
lap_time = 0.003992795944213867 [sec]
appx_time = 0.05120420455932617 [sec]
exact_time = 1.6307060718536377 [sec]
----------
lap_time = 0.0038428306579589844 [sec]
appx_time = 0.05157923698425293 [sec]
exact_time = 1.6621780395507812 [sec]
----------
